In [1]:
##libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys
import glob
from sklearn import preprocessing
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix 
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.manifold import TSNE
from colour import Color
from sklearn.covariance import EllipticEnvelope

In [2]:
import sklearn.linear_model
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.datasets import make_moons, make_blobs
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import make_pipeline

In [3]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix 
from psynlig import plot_correlation_heatmap
import pingouin as pg
from scipy.stats import pearsonr
import plotly.express as px

C:\Users\MiguelW12\PycharmProjects\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [4]:
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import chart_studio as cs
import numpy as np
import pandas as pd
import scipy
import chart_studio.plotly as py
from plotly.offline import iplot
import pandas as pd
import numpy as np  
%matplotlib inline  
from plotly import __version__ 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
import chart_studio.plotly as py
from sklearn import preprocessing
from pandas.plotting import scatter_matrix 
from psynlig import plot_correlation_heatmap
import pingouin as pg
from scipy.stats import pearsonr
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression

In [ ]:
# upload data

In [ ]:
all_files = glob.glob('gsd1a_pheno_final/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)

    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)

In [8]:
path = ('RESULTS_gsd1a_HCA_clean') 
if not os.path.exists(path):
    os.mkdir(path)
    print('Output folder created')

In [ ]:
CLEAN_SCAT_HEAT_DATA = data.copy()
##drop non-relevant feat
CLEAN_SCAT_HEAT_DATA = CLEAN_SCAT_HEAT_DATA.drop(columns=['Plane', 'Timepoint', 'Number of Analyzed Fields', 'Time [s]', 'Temperature', 'Target Temperature', 'CO2', 'Target CO2',  'Row', 'Column',
                                                         ])
##indexing
CLEAN_SCAT_HEAT_DATA["group"] = CLEAN_SCAT_HEAT_DATA["Cell Type"].astype(str) + CLEAN_SCAT_HEAT_DATA["Compound"].astype(str)
CLEAN_SCAT_HEAT_DATA.set_index(["group"], inplace = True,
                            append = True, drop = False)
CLEAN_SCAT_HEAT_DATA["group_with_id"] = CLEAN_SCAT_HEAT_DATA["CELL ID"].astype(str) + CLEAN_SCAT_HEAT_DATA["Cell Type"].astype(str) + CLEAN_SCAT_HEAT_DATA["Compound"].astype(str)
CLEAN_SCAT_HEAT_DATA.set_index(["group_with_id"], inplace = True,
                            append = True, drop = False)
CLEAN_SCAT_HEAT_DATA["group_with_pc"] = CLEAN_SCAT_HEAT_DATA["Cell Type"].astype(str) + CLEAN_SCAT_HEAT_DATA["Compound"].astype(str) + CLEAN_SCAT_HEAT_DATA["PC"].astype(str)
CLEAN_SCAT_HEAT_DATA.set_index(["group_with_pc"], inplace = True,
                            append = True, drop = False)
CLEAN_SCAT_HEAT_DATA["group_id_pc"] = CLEAN_SCAT_HEAT_DATA["group_with_id"].astype(str) + CLEAN_SCAT_HEAT_DATA["PC"].astype(str)
CLEAN_SCAT_HEAT_DATA.set_index(["group_id_pc"], inplace = True,
                            append = True, drop = False)


In [ ]:
# clear low and high cell count
cell_count =  CLEAN_SCAT_HEAT_DATA["All Cells - Number of Objects"].values
sns.histplot(cell_count)
plt.show()
CLEAN_SCAT_HEAT_DATA = CLEAN_SCAT_HEAT_DATA.loc[cell_count<1200]
sns.histplot(CLEAN_SCAT_HEAT_DATA["All Cells - Number of Objects"].values)
plt.show()
CLEAN_SCAT_HEAT_DATA = CLEAN_SCAT_HEAT_DATA.loc[cell_count>100]
sns.histplot(CLEAN_SCAT_HEAT_DATA["All Cells - Number of Objects"].values)
plt.show()

In [17]:
file_1 =   'df1_ORIGINAL_det_GSD1A.csv'
CLEAN_SCAT_HEAT_DATA.to_csv(file_1)

In [ ]:
CLEAN_SCAT_HEAT_DATA["group_with_pc"].value_counts()

In [ ]:
# OUTLIER DETECTION MODE 
DataFrame_OutliersDetections = CLEAN_SCAT_HEAT_DATA.copy()
types = CLEAN_SCAT_HEAT_DATA["group_with_pc"].unique()
mask = []
features = CLEAN_SCAT_HEAT_DATA.columns[:158]
print(types)
# detector_list = ['pca_approx', 'robust_cov']


detector_list = [
    
        (
        "Local Outlier Factor 10",
        LocalOutlierFactor(n_neighbors=10),
    ),
  

In [ ]:
for name, algorithm in detector_list:
    errors = np.full(len(DataFrame_OutliersDetections),fill_value=np.nan)
    outliers = np.full(len(DataFrame_OutliersDetections),fill_value=np.nan)

    for type in types:
        x = DataFrame_OutliersDetections.loc[:,features].values
        F = x.sum(1)
        mask = np.zeros(x.shape[0])
        mask[np.isfinite(F)] = 1
        mask_type = mask * np.array(DataFrame_OutliersDetections["group_with_pc"] == type)
        Curr_df = DataFrame_OutliersDetections.loc[mask_type==1,features]

        x = Curr_df.values
        if name == 'pca_approx':

            x = StandardScaler().fit_transform(x)
            # F = x.sum(1)
            # x = x[np.isfinite(F),:]
            # mask[1-np.isfinite(F)] = False
            # pd.DataFrame(data = Curr_df, columns = features).head()
            # pca = PCA(n_components=2)
            lower_dimensional_data = algorithm.fit_transform(x)
            pproximation = algorithm.inverse_transform(lower_dimensional_data)

            err = np.linalg.norm(x-pproximation,2,axis=1)

            errors[mask_type==1] = err
            outliers[mask_type==1] = (err < 5) * 2 - 1
            if False:
                plt.scatter(lower_dimensional_data[:,0],lower_dimensional_data[:,1],c=err>5)
                plt.title(type)
                plt.show()
        else:
            algorithm.fit(x)
            if name == "Robust covariance":
                errors[mask_type==1] = algorithm.mahalanobis(x)
                outliers[mask_type==1] = algorithm.predict(x)

            if  "Local Outlier Factor" in name:
                errors[mask_type==1] = algorithm.negative_outlier_factor_
                outliers[mask_type==1] = algorithm.fit_predict(x)
            else:
                y_pred = algorithm.fit(x).predict(x)



        # Curr_df[name] = errors[mask_type==1]
        # Curr_df[f'{name}_outliers'] = outliers[mask_type==1]
        # Curr_df.set_index(name, inplace = True,
        #                         append = True, drop = False)
        # file =   f'{type}_df_outliers_det.csv'
        # Curr_df.to_csv(file)
    DataFrame_OutliersDetections[name] = errors
    DataFrame_OutliersDetections[f'{name}_outliers'] = outliers
    DataFrame_OutliersDetections.set_index(name, inplace = True,
                            append = True, drop = False)

    file2 =   'df2_outliers_det_GSD1A.csv'
    DataFrame_OutliersDetections.to_csv(file2)


In [ ]:
DataFrame_OutliersDetections["group_with_pc"].value_counts()

In [ ]:
## cut outliers
df_clean = DataFrame_OutliersDetections[DataFrame_OutliersDetections['Local Outlier Factor 10_outliers'] == 1]
df_clean

In [ ]:
df_clean["group_with_pc"].value_counts()

In [28]:
file_3 =   'df3_outliers_det_clean_GSD1A.csv'
df_clean.to_csv(file_3)

In [33]:
# CREATE PLATE CONDITION DATA FRAMES
df_24 = df_clean[df_clean['PC'] == 24]
df_48 = df_clean[df_clean['PC'] == 48]
df_72 = df_clean[df_clean['PC'] == 72]
#define hc +hc mean data frames 
#24
hc_24 = df_24[(df_24['group'] == 'HCNONE')]
hc_24_mean = hc_24.mean()
#48
hc_48 = df_48[(df_48['group'] == 'HCNONE')]
hc_48_mean = hc_48.mean()
#72
hc_72 = df_72[(df_72['group'] == 'HCNONE')]
hc_72_mean = hc_72.mean()
# normalize to HC all conditions 
N_df_24 = df_24/hc_24_mean
N_df_48 = df_48/hc_48_mean
N_df_72 = df_72/hc_72_mean

In [36]:
#initialze the excel writer
writer = pd.ExcelWriter(path + '//' + 'livesummary-excel_full_feat.xlsx', engine='xlsxwriter')

    #store dataframes in a  dict, where the key is the sheet name 
frames = {'24': N_df_24, '48': N_df_48,
              '72': N_df_72}
    
    #now loop thru and put each on a specific sheet
for sheet, frame in  frames.items(): 
    frame.to_excel(writer, sheet_name = sheet)

    
writer.save()
    #writer.close()
print('done')


done


In [ ]:
## feature selection

In [29]:
df_clean.fillna(0, inplace=True)

In [ ]:
#lyso

In [30]:
df_lyso_a1 = df_clean[['modified_spots_chanel_4 - Total Spot Area - Mean per Well',

'modified_spots_chanel_4 - Number of Spots per Area of modified_spots_chanel_4 - Mean per Well',
'spots_chanel_4_final - Spot Area [pxֲ²] - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Area [ֲµmֲ²] - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Roundness - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Width [ֲµm] - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Length [ֲµm] - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Threshold Compactness 30%  - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Threshold Compactness 40%  - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Threshold Compactness 50%  - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Threshold Compactness 60%  - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Radial Mean  - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Radial Relative Deviation  - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Profile 1/2  - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Profile 2/2  - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Threshold Compactness 30% SER-Spot - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Threshold Compactness 40% SER-Spot - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Threshold Compactness 50% SER-Spot - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Threshold Compactness 60% SER-Spot - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Radial Mean SER-Spot - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Radial Relative Deviation SER-Spot - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Radial Mean Ratio SER-Spot - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Profile 1/2 SER-Spot - Mean per Well',
'spots_chanel_4_final - spots_chanel_4_final Profile 2/2 SER-Spot - Mean per Well',
]]
df_lyso_a1.fillna(0, inplace=True)
df_lyso_a1.to_csv(path +  '//' + 'lyso_area1_feat.csv' )
df_lyso_a1.shape

(484, 24)

In [31]:
df_lyso_a2 = df_clean[[
'modified_ir_chanel_4_ct - modified_ir_chanel_4_ct Area [ֲµmֲ²] - Mean per Well',
'modified_ir_chanel_4_ct - modified_ir_chanel_4_ct Roundness - Mean per Well',
'modified_ir_chanel_4_ct - modified_ir_chanel_4_ct Width [ֲµm] - Mean per Well',
'modified_ir_chanel_4_ct - modified_ir_chanel_4_ct Length [ֲµm] - Mean per Well',
'modified_ir_chanel_4_at - modified_ir_chanel_4_at Area [ֲµmֲ²] - Mean per Well',
'modified_ir_chanel_4_at - modified_ir_chanel_4_at Roundness - Mean per Well',
'modified_ir_chanel_4_at - modified_ir_chanel_4_at Width [ֲµm] - Mean per Well',
'modified_ir_chanel_4_at - modified_ir_chanel_4_at Length [ֲµm] - Mean per Well',
'ir_chanel_4_total - ir_chanel_4_total Area [ֲµmֲ²] - Sum per Well',
'total_ir_chanel_4_normalized']]
df_lyso_a2.fillna(0, inplace=True)
df_lyso_a2.to_csv(path +  '//' + 'lyso_area2_feat.csv' )
df_lyso_a2.shape

(484, 10)

In [32]:
df_lyso_a3 = df_clean[[
'All Cells - chanel_4 Threshold Compactness 30%  - Mean per Well',
'All Cells - chanel_4 Threshold Compactness 40%  - Mean per Well',
'All Cells - chanel_4 Threshold Compactness 50%  - Mean per Well',
'All Cells - chanel_4 Threshold Compactness 60%  - Mean per Well',
'All Cells - chanel_4 Profile 1/5  - Mean per Well',
'All Cells - chanel_4 Profile 2/5  - Mean per Well',
'All Cells - chanel_4 Profile 3/5  - Mean per Well',
'All Cells - chanel_4 Profile 4/5  - Mean per Well',
'All Cells - chanel_4 Profile 5/5  - Mean per Well',
'ir_chanel_4_total - ir_chanel_4_total Area [ֲµmֲ²] - Sum per Well',
'total_ir_chanel_4_normalized']]
df_lyso_a3.fillna(0, inplace=True)
df_lyso_a3.to_csv(path +  '//' + 'lyso_area3_feat.csv' )
df_lyso_a3.shape

(484, 11)

In [33]:
df_lyso_i = df_clean[[
'All Cells - chanel_4_intensity Mean - Mean per Well',
'All Cells - chanel_4_intensity Sum - Sum per Well',
'spots_chanel_4_final - Relative Spot Intensity - Mean per Well',
'spots_chanel_4_final - Corrected Spot Intensity - Mean per Well',
'spots_chanel_4_final - Uncorrected Spot Peak Intensity - Mean per Well',
'spots_chanel_4_final - Spot Contrast - Mean per Well',
'spots_chanel_4_final - Spot Background Intensity - Mean per Well',
'spots_chanel_4_final - Region Intensity - Mean per Well',
'spots_chanel_4_final - Spot to Region Intensity - Mean per Well']]
df_lyso_i.fillna(0, inplace=True)
df_lyso_i.to_csv(path +  '//' + 'lyso_intensity_feat.csv' )
df_lyso_i.shape

(484, 9)

In [34]:
df_lyso_t = df_clean[['All Cells - chanel_4 Gabor Max 2 px w2 - Mean per Well',
'All Cells - chanel_4 SER Spot 1 px - Mean per Well',
'All Cells - chanel_4 SER Hole 1 px - Mean per Well',
'All Cells - chanel_4 SER Ridge 1 px - Mean per Well',
'All Cells - chanel_4 SER Valley 1 px - Mean per Well']]
df_lyso_t.fillna(0, inplace=True)
df_lyso_t.to_csv(path +  '//' + 'lyso_texture_feat.csv' )
df_lyso_t.shape

(484, 5)

In [ ]:
#Tmre

In [35]:
df_tmre_a1 = df_clean[['modified_spots_chanel_3 - Total Spot Area - Mean per Well',

'modified_spots_chanel_3 - Number of Spots per Area of modified_spots_chanel_3 - Mean per Well',
'spots_chanel_3_final - Spot Area [pxֲ²] - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Area [ֲµmֲ²] - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Roundness - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Width [ֲµm] - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Length [ֲµm] - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Threshold Compactness 30%  - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Threshold Compactness 40%  - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Threshold Compactness 50%  - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Threshold Compactness 60%  - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Radial Mean  - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Radial Relative Deviation  - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Profile 1/2  - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Profile 2/2  - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Threshold Compactness 30% SER-Spot - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Threshold Compactness 40% SER-Spot - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Threshold Compactness 50% SER-Spot - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Threshold Compactness 60% SER-Spot - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Radial Mean SER-Spot - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Radial Relative Deviation SER-Spot - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Radial Mean Ratio SER-Spot - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Profile 1/2 SER-Spot - Mean per Well',
'spots_chanel_3_final - spots_chanel_3_final Profile 2/2 SER-Spot - Mean per Well']]
df_tmre_a1.fillna(0, inplace=True)
df_tmre_a1.to_csv(path +  '//' + 'tmre_area1_feat.csv' )
df_tmre_a1.shape

(484, 24)

In [36]:
df_tmre_a2 = df_clean[[
'modified_ir_chanel_3_ct - modified_ir_chanel_3_ct Area [ֲµmֲ²] - Mean per Well',
'modified_ir_chanel_3_ct - modified_ir_chanel_3_ct Roundness - Mean per Well',
'modified_ir_chanel_3_ct - modified_ir_chanel_3_ct Width [ֲµm] - Mean per Well',
'modified_ir_chanel_3_ct - modified_ir_chanel_3_ct Length [ֲµm] - Mean per Well',
'modified_ir_chanel_3_at - modified_ir_chanel_3_at Area [ֲµmֲ²] - Mean per Well',
'modified_ir_chanel_3_at - modified_ir_chanel_3_at Roundness - Mean per Well',
'modified_ir_chanel_3_at - modified_ir_chanel_3_at Width [ֲµm] - Mean per Well',
'modified_ir_chanel_3_at - modified_ir_chanel_3_at Length [ֲµm] - Mean per Well',
'ir_chanel_3_total - ir_chanel_3_total Area [ֲµmֲ²] - Sum per Well',
'total_ir_chanel_3_normalized']]
df_tmre_a2.fillna(0, inplace=True)
df_tmre_a2.to_csv(path +  '//' + 'tmre_area2_feat.csv' )
df_tmre_a2.shape

(484, 10)

In [37]:
df_tmre_a3 = df_clean[[
'All Cells - chanel_3 Threshold Compactness 30%  - Mean per Well',
'All Cells - chanel_3 Threshold Compactness 40%  - Mean per Well',
'All Cells - chanel_3 Threshold Compactness 50%  - Mean per Well',
'All Cells - chanel_3 Threshold Compactness 60%  - Mean per Well',
'All Cells - chanel_3 Profile 1/5  - Mean per Well',
'All Cells - chanel_3 Profile 2/5  - Mean per Well',
'All Cells - chanel_3 Profile 3/5  - Mean per Well',
'All Cells - chanel_3 Profile 4/5  - Mean per Well',
'All Cells - chanel_3 Profile 5/5  - Mean per Well',
'ir_chanel_3_total - ir_chanel_3_total Area [ֲµmֲ²] - Sum per Well',
'total_ir_chanel_3_normalized']]
df_tmre_a3.fillna(0, inplace=True)
df_tmre_a3.to_csv(path +  '//' + 'tmre_area3_feat.csv' )
df_tmre_a3.shape

(484, 11)

In [38]:
df_tmre_i = df_clean[[
'All Cells - chanel_3_intensity Mean - Mean per Well',
'All Cells - chanel_3_intensity Sum - Sum per Well',
'spots_chanel_3_final - Relative Spot Intensity - Mean per Well',
'spots_chanel_3_final - Corrected Spot Intensity - Mean per Well',
'spots_chanel_3_final - Uncorrected Spot Peak Intensity - Mean per Well',
'spots_chanel_3_final - Spot Contrast - Mean per Well',
'spots_chanel_3_final - Spot Background Intensity - Mean per Well',
'spots_chanel_3_final - Region Intensity - Mean per Well',
'spots_chanel_3_final - Spot to Region Intensity - Mean per Well']]
df_tmre_i.fillna(0, inplace=True)
df_tmre_i.to_csv(path +  '//' + 'tmre_intensity_feat.csv' )
df_tmre_i.shape

(484, 9)

In [39]:
df_tmre_t = df_clean[['All Cells - chanel_3 Gabor Max 2 px w2 - Mean per Well',
'All Cells - chanel_3 SER Spot 1 px - Mean per Well',
'All Cells - chanel_3 SER Hole 1 px - Mean per Well',
'All Cells - chanel_3 SER Ridge 1 px - Mean per Well',
'All Cells - chanel_3 SER Valley 1 px - Mean per Well']]
df_tmre_t.fillna(0, inplace=True)
df_tmre_t.to_csv(path +  '//' + 'tmre_texture_feat.csv' )
df_tmre_t.shape

(484, 5)

In [ ]:
#Nuc

In [40]:
df_nuc_a1 = df_clean[['All Cells - chanel_1 Symmetry 02  - Mean per Well',
'All Cells - chanel_1 Symmetry 03  - Mean per Well',
'All Cells - chanel_1 Symmetry 04  - Mean per Well',
'All Cells - chanel_1 Symmetry 05  - Mean per Well',
'All Cells - chanel_1 Symmetry 12  - Mean per Well',
'All Cells - chanel_1 Symmetry 13  - Mean per Well',
'All Cells - chanel_1 Symmetry 14  - Mean per Well',
'All Cells - chanel_1 Symmetry 15  - Mean per Well',
'All Cells - chanel_1 Axial Small Length  - Mean per Well',
'All Cells - chanel_1 Axial Length Ratio  - Mean per Well',
'All Cells - chanel_1 Radial Mean  - Mean per Well',
'All Cells - chanel_1 Radial Relative Deviation  - Mean per Well']]
df_nuc_a1.fillna(0, inplace=True)
df_nuc_a1.to_csv(path +  '//' + 'nuc_area1_feat.csv' )
df_nuc_a1.shape

(484, 12)

In [41]:
df_nuc_a2 = df_clean[[
'All Cells - chanel_1 Area [ֲµmֲ²] - Mean per Well',
'All Cells - chanel_1 Roundness - Mean per Well',
'All Cells - chanel_1 Width [ֲµm] - Mean per Well',
'All Cells - chanel_1 Length [ֲµm] - Mean per Well']]
df_nuc_a2.fillna(0, inplace=True)
df_nuc_a2.to_csv(path +  '//' + 'nuc_area2_feat.csv' )
df_nuc_a2.shape

(484, 4)

In [42]:
df_nuc_t = df_clean[['All Cells - chanel_1 Gabor Max 2 px w2 - Mean per Well', 
'All Cells - chanel_1 SER Spot 1 px - Mean per Well',
'All Cells - chanel_1 SER Hole 1 px - Mean per Well',
'All Cells - chanel_1 SER Ridge 1 px - Mean per Well',
'All Cells - chanel_1 SER Valley 1 px - Mean per Well']]
df_nuc_t.fillna(0, inplace=True)
df_nuc_t.to_csv(path +  '//' + 'nuc_texture_feat.csv' )
df_nuc_t.shape

(484, 5)

In [ ]:
#Calcien

In [43]:
df_calc_a1 = df_clean[[
'All Cells - chanel_2 Symmetry 02  - Mean per Well',
'All Cells - chanel_2 Symmetry 03  - Mean per Well',
'All Cells - chanel_2 Symmetry 04  - Mean per Well',
'All Cells - chanel_2 Symmetry 05  - Mean per Well',
'All Cells - chanel_2 Symmetry 12  - Mean per Well',
'All Cells - chanel_2 Symmetry 13  - Mean per Well',
'All Cells - chanel_2 Symmetry 14  - Mean per Well',
'All Cells - chanel_2 Symmetry 15  - Mean per Well',
'All Cells - chanel_2 Axial Small Length  - Mean per Well',
'All Cells - chanel_2 Axial Length Ratio  - Mean per Well',
'All Cells - chanel_2 Radial Mean  - Mean per Well',
'All Cells - chanel_2 Radial Relative Deviation  - Mean per Well']]
df_calc_a1.fillna(0, inplace=True)
df_calc_a1.to_csv(path +  '//' + 'calc_area1_feat.csv' )
df_calc_a1.shape

(484, 12)

In [44]:
df_calc_a2 = df_clean[['All Cells - chanel_2 Area [ֲµmֲ²] - Mean per Well',
'All Cells - chanel_2 Roundness - Mean per Well',
'All Cells - chanel_2 Width [ֲµm] - Mean per Well',
'All Cells - chanel_2 Length [ֲµm] - Mean per Well']]
df_calc_a2.fillna(0, inplace=True)
df_calc_a2.to_csv(path +  '//' + 'calc_area2_feat.csv' )
df_calc_a2.shape

(484, 4)

In [45]:
df_calc_t = df_clean[['All Cells - chanel_2 Gabor Max 2 px w2 - Mean per Well',
'All Cells - chanel_2 SER Spot 1 px - Mean per Well',
'All Cells - chanel_2 SER Hole 1 px - Mean per Well',
'All Cells - chanel_2 SER Ridge 1 px - Mean per Well',
'All Cells - chanel_2 SER Valley 1 px - Mean per Well']]
df_calc_t.fillna(0, inplace=True)
df_calc_t.to_csv(path +  '//' + 'calc_texture_feat.csv' )
df_calc_t.shape

(484, 5)

In [ ]:
path = ('Results_GSD1a_HCA_back')
if not os.path.exists(path):
    os.mkdir(path)
    print('Output folder created')
input_path = [f for f in glob.glob(os.path.join('GSD1a_feat', '*.csv'))]
print(input_path) 
verbose = False
for path_i in input_path: 
    df = pd.read_csv(path_i, skip_blank_lines=True)
    path_i.split('.csv')[0].split('GSD1a_feat')[1]
    name = path_i.split('.csv')[0].split('GSD1a_feat\\')[1]
    print('Current file name {}'.format(name))
    curr_out_path = path + '//' + '{}'.format(name)
    if not os.path.exists(curr_out_path):
        os.mkdir(curr_out_path)
        print('Output folder created')
    # read data and clean
    
    print(df.shape)
   
    
#     
    df = df.replace({'HCNONE': 1, 'GSD1ANONE':2 })
    X = df.drop(['group', 'group_with_id', 'group_with_pc', 'group_id_pc', 
                'Local Outlier Factor 10', 'Unnamed: 0'], axis=1)
    print(X.shape)
    y = df['group']
    lreg = LinearRegression()
    sfs1 = sfs(lreg, k_features=2, forward=False, verbose=1, scoring='neg_mean_squared_error')
    sfs1 = sfs1.fit(X, y)
    feat_names = list(sfs1.k_feature_names_)
    new_data = df[feat_names]
    new_data['group'] = df['group']
    new_data.to_csv(curr_out_path + '//' + '{}_selected_feat.csv'.format(name))


In [25]:
#keep features by feature selection
df_clean_final = df_clean[['All Cells - chanel_2 Symmetry 03  - Mean per Well', 'All Cells - chanel_2 Roundness - Mean per Well', 'All Cells - chanel_2 Symmetry 04  - Mean per Well', 
                           'All Cells - chanel_2 SER Spot 1 px - Mean per Well', 'All Cells - chanel_2_intensity Mean - Mean per Well',
                           'All Cells - chanel_1 Symmetry 05  - Mean per Well', 'All Cells - chanel_1 Radial Relative Deviation  - Mean per Well', 
                           'All Cells - chanel_1 SER Spot 1 px - Mean per Well', 'All Cells - chanel_1_intensity Mean - Mean per Well',
                           'spots_chanel_4_final - spots_chanel_4_final Radial Mean Ratio SER-Spot - Mean per Well', 'All Cells - chanel_4 Threshold Compactness 50%  - Mean per Well', 'modified_ir_chanel_4_ct - modified_ir_chanel_4_ct Roundness - Mean per Well',
                           'All Cells - chanel_4 SER Spot 1 px - Mean per Well', 'All Cells - chanel_4 SER Hole 1 px - Mean per Well',
                           'All Cells - chanel_4_intensity Mean - Mean per Well',
                           'All Cells - chanel_3 Profile 5/5  - Mean per Well', 'total_ir_chanel_3_normalized', 'spots_chanel_3_final - spots_chanel_3_final Radial Mean SER-Spot - Mean per Well',  
                           'spots_chanel_3_final - Region Intensity - Mean per Well', 
                           'All Cells - chanel_3 SER Hole 1 px - Mean per Well', 'All Cells - chanel_3 SER Valley 1 px - Mean per Well',
                            'group', 'group_with_id', "group_with_pc", "PC"]]

In [ ]:
#rename features

In [26]:
df_clean_final = df_clean_final.rename(
     columns={'All Cells - chanel_2 Symmetry 03  - Mean per Well': 'calc_area1', 'All Cells - chanel_2 Roundness - Mean per Well': 'calc_area2', 'All Cells - chanel_2 Symmetry 04  - Mean per Well': 'calc_area3', 'All Cells - chanel_2 Length [ֲµm] - Mean per Well': 'calc_area4',
              'All Cells - chanel_2 SER Spot 1 px - Mean per Well': 'calc_text1', 'All Cells - chanel_2_intensity Mean - Mean per Well': 'calc_intensity',
              'All Cells - chanel_1 Symmetry 05  - Mean per Well': 'nuc_area1', 'All Cells - chanel_1 Radial Relative Deviation  - Mean per Well': 'nuc_area2', 'All Cells - chanel_1 Area [ֲµmֲ²] - Mean per Well': 'nuc_area3', 'All Cells - chanel_1 Width [ֲµm] - Mean per Well': 'nuc_area4',
              'All Cells - chanel_1 SER Spot 1 px - Mean per Well': 'nuc_text1', 'All Cells - chanel_1_intensity Mean - Mean per Well': 'nuc_intensity',
              'All Cells - chanel_4 Threshold Compactness 50%  - Mean per Well': 'lyso_area1', 'spots_chanel_4_final - spots_chanel_4_final Radial Mean Ratio SER-Spot - Mean per Well': 'lyso_area2', 'modified_ir_chanel_4_ct - modified_ir_chanel_4_ct Roundness - Mean per Well': 'lyso_area3',
              'All Cells - chanel_4 SER Hole 1 px - Mean per Well': 'lyso_text1', 'All Cells - chanel_4 SER Spot 1 px - Mean per Well': 'lyso_text2', 'All Cells - chanel_4_intensity Mean - Mean per Well': 'lyso_intensity1', 
              'total_ir_chanel_3_normalized': 'tmre_area1', 'All Cells - chanel_3 Profile 5/5  - Mean per Well': 'tmre_area2', 'spots_chanel_3_final - spots_chanel_3_final Radial Mean SER-Spot - Mean per Well': 'tmre_area3',
              'All Cells - chanel_3 SER Hole 1 px - Mean per Well': 'tmre_text1', 'All Cells - chanel_3 SER Valley 1 px - Mean per Well': 'tmre_text2', 'spots_chanel_3_final - Region Intensity - Mean per Well': 'tmre_intensity1' 
             })

In [77]:
file_cor = 'df_clean_feat_GSD1A.csv'
df_clean_final.to_csv(file_cor)

In [ ]:
## export for statistics

In [29]:
# CREATE PLATE CONDITION DATA FRAMES
df_24 = df_clean_final[df_clean_final['PC'] == 24]
df_48 = df_clean_final[df_clean_final['PC'] == 48]
df_72 = df_clean_final[df_clean_final['PC'] == 72]
#define hc +hc mean data frames 
#24
hc_24 = df_24[(df_24['group'] == 'HCNONE')]
hc_24_mean = hc_24.mean()
#48
hc_48 = df_48[(df_48['group'] == 'HCNONE')]
hc_48_mean = hc_48.mean()
#72
hc_72 = df_72[(df_72['group'] == 'HCNONE')]
hc_72_mean = hc_72.mean()
# normalize to HC all conditions 
N_df_24 = df_24/hc_24_mean
N_df_48 = df_48/hc_48_mean
N_df_72 = df_72/hc_72_mean


In [32]:
#initialze the excel writer
writer = pd.ExcelWriter(path + '//' + 'n-livesummary-excel.xlsx', engine='xlsxwriter')

    #store dataframes in a  dict, where the key is the sheet name 
frames = {'24': N_df_24, '48': N_df_48,
              '72': N_df_72}
    
    #now loop thru and put each on a specific sheet
for sheet, frame in  frames.items(): 
    frame.to_excel(writer, sheet_name = sheet)

    
writer.save()
    #writer.close()
print('done')


done


C:\Users\MiguelW12\AppData\Local\Temp\ipykernel_17164\3657344344.py:13: FutureWarning:

save is not part of the public API, usage can give unexpected results and will be removed in a future version



In [ ]:
#combine df's outside and read again 
all_files = glob.glob('gsd1a_stats/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df

In [10]:
df = df.replace({'HCNONE72': 'HC72', 'GSD1ANONE72': 'GSD1A72', 
                'HCNONE48':'HC48', 'GSD1ANONE48': 'GSD1A48', 
                'HCNONE24' : 'HC24', 'GSD1ANONE24': 'GSD1A24', 
                'HCNONE': 'HC', 'GSD1ANONE': 'GSD1A'})

In [ ]:
vis = df.copy()
vis = vis.drop(columns=['Unnamed: 0'])
vis.set_index(["group"], inplace = True,
                            append = True, drop = False)

In [ ]:
# visualize each feat and each frame and move on

In [ ]:
def dist_check(df):
    
        for col in df:
            #Nh_df_24 = Nh_df_24.drop(columns=["PC", "group_with_pc", "group", "group_with_id"])
            plt.hist(df[col])
            plt.title(f"Histogram of ({col})")
            plt.xlabel(f"log({col})")
            plt.ylabel("Frequency")
            plt.show()
            plt.savefig(path + '//' + f"hist{col}.pdf", dpi = 600)
            plt.close()
            print("hist_done!")
            plt.figure()
            scipy.stats.probplot(df[col], dist="norm", plot=plt)
            plt.title(f"Q-Q-{col}")
            plt.show
            plt.savefig(path + '//' + f"Q-Q{col}.pdf", dpi = 600)
            plt.close()
            print("qq_done:)")
            plt.figure()
            sns.distplot(df[col], kde = True, color ='red', bins = 30)
            plt.title(f"dist-{col}")
            plt.show
            plt.savefig(path + '//' + f"dist{col}.pdf", dpi = 600)
            plt.close()
            print("on it:)")
            print("done:)")
        

In [ ]:
dist_check(vis)

In [ ]:
### box plots all feat
for col in vis.columns[3:]:
    
    fig = px.box(vis, x = "group", y = col, 
                 points="all", color="group",
            notched=True)
    fig.update_xaxes(categoryorder='array', categoryarray= ['HC', 'GSD1A'])
    fig.update_traces(quartilemethod="inclusive") # or "inclusive", or "linear" by default
    fig.update_layout(
    font_family="Arial",
    font_color="Black",
    font_size=20,
    font=dict(
        family="Arial",
        size=20,
        color="Black"
    )
)

    fig.write_image(path + '//' + f"{col}box_plot_image.pdf", engine="kaleido")
    fig.show()

In [ ]:
##########################################################